We will use SQL Server Management Studio to connect to an AWS RDS SQL Server database Endpoint and perform database queries on it.

#### Install the SQL Server Management Studio
Download this software from the following link:
https://docs.microsoft.com/en-us/sql/ssms/download-sql-server-management-studio-ssms

#### Go to AWS Management Console and Launch a database instance
* Engine options
    * Under the Service RDS, Choose Engine options as Microsoft SQL Server.
    * Choose the edition SQL Server Express Edition and the latest version SQL Server 2019.
    <img src='Database_Instance.png' width='700'>
* Provide credential settings for the database
    * Choose a username and password for your database.
    <img src='Credential_Settings.png' width='700'>
* Instance class and Storage settings
    * Pick the default instance class db.t3.small and Storage type General Purpose SSD.
    * Turn off storage autoscaling.
    <img src='Instance_Size.png' width='700'>
* Finish Setup of instance
    * Choose your default VPC and Click on Create Database.

#### Connect to your database endpoint
* Once your database instance status is Available, Under the option Connectivity and Security, Copy the Endpoint Name.
<img src='Endpoint.png' width='700'>
* Go to SSMS, Click on Connect and Choose Database Engine.
    * Enter you Server Name and choose SQL Server Authentication and enter your database server credentials. Click on Connect.

#### Working on our SQL Operations
Create a new database and Import your tables and start working on your SQL Operations
* Selection or Projection
    * Our table dbo.phi_CA contains COVID-19 data for Confirmed, Deaths, Tested and Recovered Cases for each day and also cumulative data.
    <img src='Selection.png' width=700>
    <code>
    SELECT 
        [date], numconf, numdeaths, numtested, numrecover 
    FROM 
        dbo.phi_CA 
    WHERE 
        prname = 'Ontario'
    ORDER BY 
        [date] DESC;
    </code>
    <img src='Selection_2.png' width=700>
    * Select DISTINCT Provinces and Territories of Canada.
    <code>
    SELECT DISTINCT prname from dbo.phi_CA;
    </code>
    <img src='Selection_3.png' width=700>
* Filtering
    * Select Province Names WHERE Province Names begin with O, B or Q.
    <code>
    SELECT DISTINCT prname FROM dbo.phi_CA WHERE prname LIKE '[OBQ]%';
    </code>
    <img src='Filtering.png' width=700>
    * Select dates WHERE number of Confirmed COVID Cases in Ontario were between 900 and 1400.
    <code>
    SELECT 
        prname, [date], numtoday 
    FROM 
        dbo.phi_CA 
    WHERE 
        prname = 'Ontario' AND numtoday BETWEEN 900 AND 1400
    ORDER BY
        [date];
    </code>
    <img src='Filtering_2.png' width=700>
* Joins
    * We have a new table phi_JH containing COVID-19 data for Canada but it does not contain daily statistics. We will join this table with phi_CA table to retrieve COVID-19 daily statistics.
    <img src='Joins.png' width=700>
    * Defining a Primary Key Constraint on phi_JH table using Composite key (Date, Province)
    <code>
    ALTER TABLE 
        dbo.phi_JH
    ADD CONSTRAINT 
        PK_phi_JH_Date_Province 
    PRIMARY KEY
        ([Date], Province);
    </code>
    * Defining a Foreign Key Constraint on phi_CA table REFERENCING dbo_JH table columns Date, Province
    <code>
    ALTER TABLE 
        dbo.phi_CA
    ADD CONSTRAINT 
        FK_phi_CA_date_prname
    FOREIGN KEY
        ([date], prname)
    REFERENCES 
        dbo.phi_JH ([Date], Province);
    </code>
    * Finding COVID-19 Daily Statistics Using INNER JOIN Operation between dbo.phi_CA and dbo.phi_JH tables .
    <code>
    SELECT 
        dbo.phi_JH.[Date], Province, numtoday, numdeathstoday, numtestedtoday, numrecoveredtoday
    FROM 
        dbo.phi_CA
    INNER JOIN
        dbo.phi_JH
    ON 
        dbo.phi_CA.[date] = dbo.phi_JH.[Date] AND dbo.phi_CA.prname = dbo.phi_JH.Province
    ORDER BY
        Province, dbo.phi_JH.[Date];
    </code>
    <img src='Joins_2.png' width=700>
* Aggregations
    * Getting the total number of Confirmed Cases of COVID-19 for each Province by using GROUP BY Clause in conjunction with Aggregate Function SUM.
    <code>
    SELECT 
        prname AS 'Province/Territory', 
        SUM(numtoday) AS 'Confirmed Cases' 
    FROM 
        dbo.phi_CA 
    GROUP BY 
        prname
    ORDER BY
        SUM(numtoday) DESC;
    </code>
    <img src='Aggregations.png' width=700>
* Sorting
    * Sort the COVID-19 Data for Canada where Province Name is Sorted in ORDER BY of ASC Order OR Lowest to Highest and Number of Recoveries are Sorted in Order of DESC OR Highest to Lowest for each Date.
    <code>
    SELECT 
        prname AS 'Province/Territory', 
        [date] AS 'Date',
        numrecoveredtoday AS 'Number of Recoveries' 
    FROM 
        dbo.phi_CA 
    ORDER BY
        prname ASC,
        numrecoveredtoday DESC;
    </code>
    <img src='Sorting.png' width='700'>